In [1]:
#Done
from hazm import *

In [2]:
#Done
import os
import math

In [3]:
#Done
def get_stopwords(directory):
    stopwords = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as f:
            for stopword in f.read().splitlines():
                if stopword not in stopwords:
                    stopwords.append(stopword)
    return stopwords        

In [258]:
#Done
def prepare_text(raw_text):
    prepared_text = []
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(raw_text)
    tokenized_text = word_tokenize(normalized_text)
    stopwords = get_stopwords('stopwords')
    stemmer = Stemmer()
    
    for token in tokenized_text:
        if token not in stopwords:
            stemmed_text = stemmer.stem(token)
            if stemmed_text != "":
                
                prepared_text.append(stemmed_text.strip())

    return prepared_text

In [5]:
#Done
def prepare_text(raw_text):
    prepared_text = []
    stopwords = get_stopwords('stopwords')
    
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(raw_text)
    for stop in stopwords:
        normalized_text = normalized_text.replace(stop, " ")
    tokenized_text = word_tokenize(normalized_text)
    stemmer = Stemmer()
    
    for token in tokenized_text:
        if token not in stopwords:
            stemmed_text = stemmer.stem(token)
            if stemmed_text != "":
                prepared_text.append(stemmed_text)

    return prepared_text

In [6]:
#Done
from xml.dom.minidom import parse

In [7]:
#Done
# Reading Data (XML file) and store it in dom variable
dom = parse('project1_data/data/Persian.xml')

In [8]:
#Done
def get_title(raw_title):
    title = raw_title[0].childNodes[0].data
    return title

In [9]:
#Done
def get_text(raw_text):
    text = raw_text[0].childNodes[0].data
    return text

In [10]:
#Done
def get_id(raw_id):
    doc_id = raw_id[0].childNodes[0].data
    return doc_id

In [11]:
#Done
import json

In [12]:
#Done
positional_index = {}
doc_length = {} #number of words for each documents

def document_length(title_words_list, text_words_list, doc_id):
    all_words = []
    for word in title_words_list:
        if word not in all_words:
            all_words.append(word)
    
    for word in text_words_list:
        if word not in all_words:
            all_words.append(word)
    doc_length[doc_id] = len(all_words)
    

def construct_positional_indexes(docs_path):
    dom = parse(docs_path)
    pages = dom.getElementsByTagName('page')    
    for page in pages:
        title = get_title(page.getElementsByTagName('title'))
        text = get_text(page.getElementsByTagName('text'))
        doc_id = get_id(page.getElementsByTagName('id'))
        
        text_words_list = prepare_text(text)
        title_words_list = prepare_text(title)
        
        document_length(title_words_list, text_words_list, doc_id)
        
        for index, text_word in enumerate(text_words_list):
            if text_word not in positional_index:
                initializer = {doc_id: {'text': []}}
                positional_index[text_word] = initializer
            elif doc_id not in positional_index[text_word]:
                positional_index[text_word][doc_id] = {'text': []}
            elif 'text' not in positional_index[text_word][doc_id]:
                positional_index[text_word][doc_id]['text'] = []
            positional_index[text_word][doc_id]['text'].append(index)

            
        for index, title_word in enumerate(title_words_list):
            if title_word not in positional_index:
                initializer = {doc_id: {'title': []}}
                positional_index[title_word] = initializer
            elif doc_id not in positional_index[title_word]:
                positional_index[title_word][doc_id] = {'title': []}
            elif 'title' not in positional_index[title_word][doc_id]:
                positional_index[title_word][doc_id]['title'] = []
            positional_index[title_word][doc_id]['title'].append(index)
    

construct_positional_indexes('project1_data/data/Persian.xml')

In [13]:
#Done
def get_posting_list(word):
    # Make sure how to get posting list! From reading file or as an argument
    return positional_index[word]

In [14]:
def get_words_with_bigram(bigram):
    # WARNING: not based on slides     
    words = []
    for word in positional_index:
        if bigram in word:
            words.append(word)
    return words
# get_words_with_bigram('لا')

In [15]:
def get_words_with_bigram(bigram):
    # WARNING: based on slides    
    inverted_index = {}
    for word in positional_index:
        word_length = len(word)
        if word_length >= 2:
            for i in range(len(word) - 2):
                bi = word[i: i + 2]
                if bi not in inverted_index:
                    inverted_index[bi] = []
                if word not in inverted_index[bi]:
                    inverted_index[bi].append(word)
    return inverted_index[bigram]
                    
# get_words_with_bigram('لا')

In [17]:
def add_document_to_indexes(docs_path, doc_num):
    address = docs_path + '/' + doc_num + '.xml'
    construct_positional_indexes(address)

# add_document_to_indexes('data/wiki', 20)

In [18]:
def delete_document_from_indexes(docs_path, doc_num):
    address = docs_path + '/' + doc_num + '.xml'
    dom = parse(address)
    pages = dom.getElementsByTagName('page')    
    for page in pages:
        title = get_title(page.getElementsByTagName('title'))
        text = get_text(page.getElementsByTagName('text'))
        doc_id = get_id(page.getElementsByTagName('id'))
        
        text_words_list = prepare_text(text)
        title_words_list = prepare_text(title)
        
        for index, text_word in enumerate(text_words_list):
            if text_word not in positional_index:
                print('کلمه پیدا نشد')
            elif doc_id not in positional_index[text_word]:
                print('چنین کلمه‌ای با این شماره سند موجود نیست')
            else:
                positional_index[text_word].remove(doc_id)
            if len(positional_index[text_word]) == 0:
                del positional_index[text_word]
                
        for index, title_word in enumerate(title_words_list):
            if title_word not in positional_index:
                print('کلمه پیدا نشد')
            elif doc_id not in positional_index[title_word]:
                print('چنین کلمه‌ای با این شماره سند موجود نیست')
            else:
                positional_index[title_word].remove(doc_id)
            if len(positional_index[title_word]) == 0:
                del positional_index[title_word]

# delete_document_from_indexes('data/wiki', 10)

In [19]:
def save_index(destination):
    full_destination = destination + '.json'
    with open(full_destination, 'w') as f:
        json.dump(positional_index, f, ensure_ascii=False)

# save_index('storage/index_backup')

In [20]:
import re

In [110]:
def get_all_queries(directory):
    queries = []
    for filename in os.listdir(directory):
        with open (os.path.join(directory, filename), 'r') as f:
            query = f.readline()
            queries.append(query)
    return queries
queries = get_all_queries('project1_data/data/queries/')
for query in queries:
    print(parsing_query(query))

([], ['طبیعت', 'دامنه', 'کوه', 'ایرانی'])
(['علوم اجتماعی'], ['مطالعه', 'در', 'دانشگاه'])
([], ['هیتلر', 'در', 'جنگ', 'جهانی', 'اول'])
(['منظومه شمسی'], ['سیاره', 'های', 'بزرگ', '\n'])
([], ['جنگل', 'های', 'بلوط', 'ایران'])
([], ['زندگی', 'حیوانات', 'وحشی'])
([], ['مسابقات', 'فوتبال', 'المپیک'])
([], ['کشورهای', 'عضو', 'اتحادیه', 'آفریقا'])
([], ['کتاب', 'های', 'برگزیده', 'کودک', 'و', 'نوجوان'])
([], ['برنده', 'جایزه', 'بهترین', 'فیلم', 'در', 'جشنواره'])
([], ['ابزار', 'های', 'فضایی', 'و', 'پیشرفته', 'ناسا'])
([], ['سواحل', 'دریای', 'سرخ'])
(['خلیج فارس'], ['پایتخت', 'کشورهای', 'حوزه', ''])
([], ['کشورهای', 'دارای', 'نفت', 'در', 'خاورمینا'])
([], ['انتخابات', 'نمایندگان', 'مجلس', 'ایالتی', 'در', 'آمریکا'])
([], ['تاریخچه', 'هنر', 'نمایشی', 'در', 'ایران'])
(['باشگاه فوتبال'], ['', 'اروپایی'])
([], ['تاریخ', 'علوم', 'اجتماعی', 'در', 'اروپا'])
([], ['جاذبه', 'گردشگری', 'در', 'استان', 'کردستان'])
([], ['درمان', 'بیماری', 'افسردگی'])


In [270]:
def positional_intersect(phrase):
    phrases = prepare_text(phrase)
    if len(phrases) == 1:
        return get_posting_list(phrases[0])
    
    phrase_touples = []
    
    for i in range(len(phrases) - 1):
        phrase_touples.append((phrases[i], phrases[i + 1]))
    
    answer = {}
    
    for index, (phrase1, phrase2) in enumerate(phrase_touples):
        doc_list1 = {}
        if index == 0:
            doc_list1 = positional_index[phrase1]
            
        elif not answer:
            return answer
        
        else:
            doc_list1 = answer
            answer = {}

        try:
            doc_list_len1 = len(doc_list1)
            doc_list2 = positional_index[phrase2]
            doc_list_len2 = len(doc_list2)

        except:
            return answer
        i, j = 0, 0
        while i < doc_list_len1 and j < doc_list_len2:
            doc1_keys = list(doc_list1.keys())
            doc2_keys = list(doc_list2.keys())


            doc1_key = doc1_keys[i]
            doc2_key = doc2_keys[j]


            if doc1_key == doc2_key:
                doc_id = doc1_key
                contexts = ['title', 'text']
                for context in contexts:
                    try:
                        context_indices1 = doc_list1[doc1_key][context]
                        context_indices2 = doc_list2[doc2_key][context]
                    except:
                        continue


                    m, n = 0, 0
                    while m < len(context_indices1) and n < len(context_indices2):
                        first_phrase_index = context_indices1[m]
                        second_phrase_index = context_indices2[n]

                        if second_phrase_index - first_phrase_index == 1:
                            if doc_id not in answer:
                                answer[doc_id] = {}

                            if context not in answer[doc_id]:
                                answer[doc_id][context] = []

                            answer[doc_id][context].append(second_phrase_index)

                            m += 1
                            n += 1
                        elif first_phrase_index < second_phrase_index:
                            m += 1
                        else:
                            n += 1
                    i += 1
                    j += 1

            elif doc1_key < doc2_key:
                i += 1
            else:
                j += 1
                
#     print(answer)
    return answer

def get_phrase_occurence(phrase): 
    #count the number of occurence of phrase in docuemnts
    documents = positional_intersect(phrase[:])
    document_occurence = len(list(documents.keys()))
    return document_occurence


In [217]:
def phrase_normalizer(double_quoted):
    new_double_quotes = []
    for quote in double_quoted:
        quote_list = prepare_text(quote)
        phrase = ""
        for quotes in quote_list:
            phrase += quotes + " "
        new_double_quotes.append(phrase.strip())
    return new_double_quotes

def parsing_query(query):
    all_quotations = re.findall('"([^"]*)"', query)
    query = query.replace('"', "")
    for quoted in all_quotations:
        query = query.replace(quoted, "")
        query = query.replace('  ', " ")
    normalized_phrases = phrase_normalizer(all_quotations[:])
    not_sequential = prepare_text(query)
    return normalized_phrases, not_sequential

In [302]:
N = len(doc_length)

def get_quoted_posting_list(phrases):
    first_double_quote = phrases.pop()
    first_double_quote_info = positional_intersect(first_double_quote)
    first_double_quote_docs = list(first_double_quote_info.keys()) #docs containing the first double quote
    answer = first_double_quote_docs
    
    for double_quote in phrases:
        double_quote_info = positional_intersect(double_quote)
        if double_quote_info:            
            other_double_quote_docs = list(double_quote_info.keys())
            new_answer = []
            for ans_doc_id in answer:
                for doc_id_other in other_double_quote_docs:
                    if ans_doc_id == doc_id_other:
                        new_answer.append(ans_doc_id)
            answer = new_answer
        else:
            return []

    return answer


def get_dictionary(query_list):
    dic = {}
    for term in query_list:
        if term not in dic:
            dic[term] = 1
        else:
            dic[term] += 1
    return dic


def consine_score(query_list, method, weight, quoted=False, double_quoted=None, contexts=['title', 'text']):
    if quoted: #new
        valid_docs = get_quoted_posting_list(double_quoted[:]) #docs that have all phrases
        
    else:
        valid_docs = list(doc_length.keys())
        
    scores = {} #score of documents
    query_terms_occurence = get_dictionary(query_list) #terms occurence in query
    terms = list(query_terms_occurence.keys()) #all distinct terms in query
    doc_normalization = 0
    
    print('terms', terms)

    for term in terms:
        if term in positional_index or term in double_quoted:
            if quoted:
                docs_list = positional_intersect(term)
            else:
                docs_list = positional_index[term]

            tf_q = query_terms_occurence[term]
            document_occurence = len(docs_list)
            print('term:', term, 'query occ:', query_terms_occurence[term], 'doc occ:', document_occurence)
            
            idf = math.log(N / document_occurence)
            w_tq = tf_q * idf

            for doc in docs_list:
                if doc in valid_docs:
                    tf = 0
                    for context in contexts:
                        if 'title' in docs_list[doc] and context == 'title':
                            title_length = len(docs_list[doc]['title'])
                            tf += weight * (1 + math.log(title_length))
                        elif 'text' in docs_list[doc] and context == 'text':
                            text_length = len(docs_list[doc]['text'])
                            tf += 1 + math.log(text_length)    
                        w_td = tf
                        doc_normalization += w_td * w_td
                        if doc in scores:
                            scores[doc] += w_tq * w_td
                        else:
                            scores[doc] = w_tq * w_td
    if method == 'ltc-lnc':
        doc_normalization = math.sqrt(doc_normalization)
        for doc in scores:
            scores[doc] /= doc_normalization  
    scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)} #TODO: not works
    return scores


def search(query, method="ltn-lnn", weight=2, contexts=['title', 'text']):
    print('query', query)
    double_quoted, not_sequential = parsing_query(query)

    all_words = [] #new
    all_words.extend(double_quoted + not_sequential) #new

    if not double_quoted:
        return consine_score(all_words[:], method, weight, contexts=contexts[:])
    
    return consine_score(all_words, method, weight, True, double_quoted[:], contexts=contexts[:])

search(queries[1], 'ltc-lnc')

query مطالعه "علوم اجتماعی" در دانشگاه
terms ['علو اجتماع', 'مطالعه', 'در', 'دانشگاه']
term: علو اجتماع query occ: 1 doc occ: 20
term: مطالعه query occ: 1 doc occ: 96
term: در query occ: 1 doc occ: 1240
term: دانشگاه query occ: 1 doc occ: 322


{'3069': 0.2557408645178948,
 '4066': 0.32556941983460935,
 '4388': 0.6677838781807702,
 '4497': 0.3239281498413287,
 '4568': 0.16128208931032467,
 '4833': 0.479255573197921,
 '4899': 0.20070264454360964,
 '5201': 0.2814089674058235,
 '5428': 0.804097782849659,
 '5449': 0.29239184936892626,
 '5571': 0.4333875765436848,
 '5872': 0.46785709709498713,
 '6506': 0.4939036577557021,
 '6589': 0.2958508036357524,
 '6634': 0.6258653344622294,
 '6694': 0.6698670241379651,
 '6791': 1.0164452807102937,
 '6907': 0.5722512968242297,
 '6973': 0.37791099742817885,
 '7002': 0.5168286121574529}

In [303]:
def detailed_search(title_query, text_query, method="ltn-lnn"):
    relevant_docs = {}
    title_score = search(title_query, method=method, weight=2, contexts=['title'])
    text_score = search(text_query, method=method, weight=1, contexts=['text'])
    #print(title_query)
    #print(desc_query)
    return title_score, text_score

detailed_search('عجایب هفت‌گانه', 'چشمگیرترین بناهای تاریخی جهان', "ltc-lnc")

query عجایب هفت‌گانه
terms ['عجایب', 'هفت\u200cگانه']
term: عجایب query occ: 1 doc occ: 13
term: هفت‌گانه query occ: 1 doc occ: 9
query چشمگیرترین بناهای تاریخی جهان
terms ['چشمگیر', 'بنا', 'تاریخ', 'جه']
term: چشمگیر query occ: 1 doc occ: 48
term: بنا query occ: 1 doc occ: 291
term: تاریخ query occ: 1 doc occ: 708
term: جه query occ: 1 doc occ: 614


({'3072': 0.0,
  '3260': 0.0,
  '3359': 0.0,
  '3414': 0.0,
  '3666': 0.0,
  '3785': 0.0,
  '3853': 0.0,
  '3854': 7.041393376592041,
  '4002': 0.0,
  '4321': 0.0,
  '4530': 0.0,
  '5236': 0.0,
  '6162': 0.0,
  '6186': 0.0,
  '6233': 0.0,
  '6589': 0.0,
  '6604': 0.0,
  '6606': 0.0,
  '6969': 0.0},
 {'3016': 0.054525775776761445,
  '3021': 0.010381992782338267,
  '3023': 0.008808870795406194,
  '3027': 0.08045375465403784,
  '3029': 0.01757824180800974,
  '3030': 0.062283513071478856,
  '3033': 0.04936673928996292,
  '3036': 0.010381992782338267,
  '3037': 0.010381992782338267,
  '3039': 0.09819197859773543,
  '3041': 0.03964833939817098,
  '3046': 0.008808870795406194,
  '3047': 0.026387112603415933,
  '3049': 0.010381992782338267,
  '3055': 0.0,
  '3056': 0.01491471475115884,
  '3058': 0.07390474469377752,
  '3065': 0.01919086357774446,
  '3068': 0.1144256502471472,
  '3069': 0.010381992782338267,
  '3072': 0.092278465269935,
  '3073': 0.06207869088855816,
  '3091': 0.024592248456281